In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import xarray as xr 
import contextily as ctx

In [ ]:
# Import perimeter of eaton fire 
fp = os.path.join("data", "Eaton_Perimeter_20250121", "Eaton_Perimeter_20250121.shp")
eaton_boundary = gpd.read_file(fp)

# Import perimeter of palisades fire 
fp = os.path.join("data", "Palisades_Perimeter_20250121", "Palisades_Perimeter_20250121.shp")
palisades_boundary = gpd.read_file(fp)

# Import CA EJI gdb 
fp = os.path.join("data", "EJI_2024_California", "EJI_2024_California.gdb")
eji_california = gpd.read_file(fp)


In [ ]:
fires = gpd.GeoDataFrame(pd.concat([eaton_boundary, palisades_boundary]))

In [ ]:
crs_reference = palisades_boundary.crs

eji_california = eji_california.to_crs(crs_reference)

eji_palisades = gpd.sjoin(eji_california, palisades_boundary, predicate = 'intersects')

In [ ]:
eji_palisades.plot()
palisades_boundary.plot()

In [ ]:
crs_reference = eaton_boundary.crs

eji_california = eji_california.to_crs(crs_reference)

eji_eaton = gpd.sjoin(eji_california, eaton_boundary, predicate = 'intersects')

In [ ]:
fig, ax = plt.subplots()

eji_palisades_clipped.plot(ax = ax)

palisades_boundary.plot(ax = ax, 
                        color = 'none', 
                        edgecolor='black')

In [ ]:
eji_palisades_clipped = gpd.clip(eji_california, palisades_boundary)

In [ ]:
eji_palisades_clipped.plot()

In [ ]:
eji_eaton_clipped = gpd.clip(eji_california, eaton_boundary)
eji_eaton_clipped.plot()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 12))

# ADD FIRE PERIMETERS: UPDATE FILL TRANSPARENCY AND COLOR

palisades_boundary.plot(ax=ax,        
                edgecolor = 'black',
                color = 'red',
                linewidth= 1.25, 
                alpha = .3)

eaton_boundary.plot(ax=ax,         
                 edgecolor='black',
                 color = 'red',
                 linewidth=1.25, 
                 alpha = .4)

# ADD LEGEND OR ANNOTATION TO IDENTIFY EACH FIRE
plt.figtext(x = .63, 
            y = .65,
            s ="Eaton Fire Boundary", 
            weight = 'bold')

plt.figtext(x = .2, 
            y = .51,
            s ="Palisades Fire Boundary", 
            weight = 'bold')
# ADD TITLE

# Add basemap using contextily

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, 
                alpha = .5)

ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# UPDATE WITH YOU EJI VARIABLE FROM STEP 1
eji_variable = 'EPL_POV200'

# Find common min/max for legend range
vmin = min(eji_palisades_clipped[eji_variable].min(), eji_eaton_clipped[eji_variable].min())
vmax = max(eji_palisades_clipped[eji_variable].max(), eji_eaton_clipped[eji_variable].max())

# Plot census tracts within Palisades perimeter
eji_palisades_clipped.plot(
    column= eji_variable,
    vmin=vmin, vmax=vmax,
    legend=False,
    ax=ax1,
    cmap = 'inferno'
)
ax1.set_title('Palisades Fire')
ax1.axis('off')

# Plot census tracts within Eaton perimeter
eji_eaton_clipped.plot(
    column=eji_variable,
    vmin=vmin, vmax=vmax,
    legend=False,
    ax=ax2,
)
ax2.set_title('Eaton Fire')
ax2.axis('off')

# Add overall title
fig.suptitle('Population 200% below the Poverty Level')

# Add shared colorbar at the bottom
sm = plt.cm.ScalarMappable( norm=plt.Normalize(vmin=vmin, vmax=vmax))
cbar_ax = fig.add_axes([0.25, 0.08, 0.5, 0.02])  # [left, bottom, width, height]
cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
cbar.set_label('Poverty Level Percentile')

plt.show()